In [1]:
import cv2
## opencv-contrib==3.4.9.33
import numpy as np

cap = cv2.VideoCapture(0)
ret, frame = cap.read()
height_f, width_f, _ = frame.shape
print(f'shape {width_f}, {height_f}')

coords = []
drawing = False
roi = None
mouse_x = None
mouse_y = None
wzor = None
gray_wzor = None
jest_wzor = False
duzy_gray_wzor = None
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',
            'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']

def filtruj(image):
    image = image.copy()
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = cv2.GaussianBlur(image,(3,3),0)
    image = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,7,5)
    image = cv2.Laplacian(image, cv2.CV_8U)
    
    #image = cv2.Canny(image,20,80)
    return image

def click_and_crop(event, x, y, flag, image):
    global coords, drawing, roi, mouse_x, mouse_y,wzor,wykryto_kontury, jest_wzor,gray_wzor,height_f, width_f,duzy_gray_wzor
    mouse_x = x
    mouse_y = y
    if event == cv2.EVENT_LBUTTONDOWN:
        # user has clicked the mouse's left button
        drawing = True
        # save those starting coordinates
        coords = [(x, y)]
    elif event == cv2.EVENT_MOUSEMOVE:
    # user is moving the mouse within the window
        if drawing:
            # if we're in drawing mode, we'll draw a green rectangle
            # from the starting x,y coords to our current coords
            clone = frame.copy()
            #cv2.rectangle(frame, coords[0], (x, y), (0, 255, 0), 2)
    elif event == cv2.EVENT_LBUTTONUP:
    # user has released the mouse button, leave drawing mode
    # and crop the photo
        drawing = False
        # save our ending coordinates
        coords.append((x, y))
        if len(coords) == 2:
            # calculate the four corners of our region of interest
            if coords[0][0] > coords[1][1]:
                first = coords[1]
                second = coords [0]
                coords[0] = first
                coords[1] = second
            ty, by, tx, bx = coords[0][1], coords[1][1], coords[0][0], coords[1][0]
            # crop the image using array slicing
            jest_wzor = False
            wzor = frame[ty:by, tx:bx]
            height, width = wzor.shape[:2]
            if width > 0 and height > 0:
                # make sure roi has height/width to prevent imshow error
                # and show the cropped image in a new window
                gray_wzor = filtruj(wzor)
                print(f'shape_gray_wzor {gray_wzor.shape[0]}, {gray_wzor.shape[1]}')
                print(f'wspolzedne {by-ty}, {bx-tx}')
                cv2.namedWindow("wzor")
                cv2.imshow("wzor", wzor)
                duzy_gray_wzor = np.zeros((height_f,width_f), np.uint8)
                print(f'shape_duzy_gray_wzor {duzy_gray_wzor.shape[0]}, {duzy_gray_wzor.shape[1]}')
                duzy_gray_wzor[ty:by, tx:bx] = gray_wzor
                duzy_gray_wzor = cv2.merge([np.ones((height_f,width_f), np.uint8),duzy_gray_wzor,np.ones((height_f,width_f), np.uint8)])
                #cv2.imshow('duzy wzor', duzy_gray_wzor)
                cv2.namedWindow("gray_wzor")
                cv2.imshow("gray_wzor", gray_wzor)
                jest_wzor = True
                
                
                
                



cv2.namedWindow('kamera')
cv2.setMouseCallback('kamera',click_and_crop)
wykryto_kontury = False;
                
while True:
    i = 0
    ret, frame = cap.read()
    
    copy_frame = frame.copy()
    gray_frame = filtruj(frame)
    color_edges = cv2.merge([gray_frame,np.ones((gray_frame.shape[0],gray_frame.shape[1]), np.uint8),np.ones((gray_frame.shape[0],gray_frame.shape[1]), np.uint8)])
    
    copy_frame = cv2.addWeighted(copy_frame, 1, color_edges, 1,0)
        
    if drawing:
        cv2.rectangle(copy_frame, coords[0], (mouse_x, mouse_y), (0, 255, 0), 2)
    if jest_wzor:
        if duzy_gray_wzor is not None:
            copy_frame = cv2.addWeighted(copy_frame, 1, duzy_gray_wzor, 1,0)
        w, h = gray_wzor.shape[::-1]
        
        
        #####################
        img = gray_frame.copy()
        ######################
        method = eval(methods[1])

        # Apply template Matching
        res = cv2.matchTemplate(img,gray_wzor,method)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
        if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
            top_left = min_loc
        else:
            top_left = max_loc
            
        bottom_right = (top_left[0] + w, top_left[1] + h)
        cv2.putText(copy_frame, f'max_val: {max_val}',(10,30), cv2.FONT_HERSHEY_SIMPLEX,0.5,(209, 80, 0, 255), 1)
        cv2.putText(gray_frame, f'max_val: {max_val}',(10,30), cv2.FONT_HERSHEY_SIMPLEX,0.5,(209, 80, 0, 255), 1)
        if max_val >= 0.20:
            cv2.rectangle(copy_frame,top_left, bottom_right, 255, 2)
            cv2.rectangle(gray_frame,top_left, bottom_right, 255, 2)

        
    cv2.imshow('kamera',copy_frame)
    cv2.imshow('edges',gray_frame)
    k = cv2.waitKey(1)
    if k == 27:
        break
    
    
    
cap.release()
cv2.destroyAllWindows()

shape 640, 480
shape_gray_wzor 213, 482
wspolzedne 213, 482
shape_duzy_gray_wzor 480, 640
